# Transaction Isolation Levels

Transaction isolation levels define the degree to which one transaction must be isolated from data modifications made by other concurrent transactions. Understanding these levels is crucial for building reliable database applications that balance **data consistency** with **performance**.

---

## Overview

The SQL standard defines **four isolation levels**, each providing different guarantees about what phenomena can occur during concurrent transaction execution:

| Isolation Level | Dirty Read | Non-Repeatable Read | Phantom Read |
|-----------------|:----------:|:-------------------:|:------------:|
| **Read Uncommitted** | ✅ Possible | ✅ Possible | ✅ Possible |
| **Read Committed** | ❌ Prevented | ✅ Possible | ✅ Possible |
| **Repeatable Read** | ❌ Prevented | ❌ Prevented | ✅ Possible |
| **Serializable** | ❌ Prevented | ❌ Prevented | ❌ Prevented |

---

## Concurrency Phenomena Explained

### 1. Dirty Read
A transaction reads data written by another **uncommitted** transaction. If that transaction rolls back, the first transaction has read "dirty" (invalid) data.

### 2. Non-Repeatable Read
A transaction reads the same row twice and gets **different values** because another transaction modified and committed the row between the two reads.

### 3. Phantom Read
A transaction re-executes a query and finds **new rows** that weren't there before, because another transaction inserted rows matching the query criteria.

---

## Python Simulation Setup

We'll simulate these isolation levels using Python classes to demonstrate the behavior of each level. This simulation mimics how databases handle concurrent transactions.

In [ ]:
import threading
import time
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Any
from enum import Enum
from copy import deepcopy
import uuid


class IsolationLevel(Enum):
    READ_UNCOMMITTED = 1
    READ_COMMITTED = 2
    REPEATABLE_READ = 3
    SERIALIZABLE = 4


@dataclass
class Row:
    """Represents a database row with versioning."""
    id: int
    data: Dict[str, Any]
    committed: bool = True
    transaction_id: Optional[str] = None


@dataclass
class Transaction:
    """Represents a database transaction."""
    id: str = field(default_factory=lambda: str(uuid.uuid4())[:8])
    isolation_level: IsolationLevel = IsolationLevel.READ_COMMITTED
    snapshot: Optional[Dict[int, Row]] = None
    read_set: List[int] = field(default_factory=list)
    write_set: List[int] = field(default_factory=list)
    active: bool = True


class SimpleDatabase:
    """
    A simplified database simulation to demonstrate isolation levels.
    """
    
    def __init__(self):
        self.tables: Dict[str, Dict[int, Row]] = {"accounts": {}}
        self.uncommitted_writes: Dict[str, Dict[int, Row]] = {}  # txn_id -> {row_id: Row}
        self.lock = threading.RLock()
        self.active_transactions: Dict[str, Transaction] = {}
    
    def insert(self, table: str, row_id: int, data: Dict[str, Any]):
        """Insert initial data (committed)."""
        with self.lock:
            self.tables[table][row_id] = Row(id=row_id, data=data, committed=True)
    
    def begin_transaction(self, isolation_level: IsolationLevel) -> Transaction:
        """Start a new transaction with the specified isolation level."""
        with self.lock:
            txn = Transaction(isolation_level=isolation_level)
            self.active_transactions[txn.id] = txn
            
            # For REPEATABLE_READ and SERIALIZABLE, take a snapshot
            if isolation_level in [IsolationLevel.REPEATABLE_READ, IsolationLevel.SERIALIZABLE]:
                txn.snapshot = deepcopy(self.tables["accounts"])
            
            return txn
    
    def read(self, txn: Transaction, table: str, row_id: int) -> Optional[Row]:
        """Read a row based on the transaction's isolation level."""
        with self.lock:
            txn.read_set.append(row_id)
            
            if txn.isolation_level == IsolationLevel.READ_UNCOMMITTED:
                # Can see uncommitted writes from other transactions
                for other_txn_id, writes in self.uncommitted_writes.items():
                    if row_id in writes:
                        return writes[row_id]
                return self.tables[table].get(row_id)
            
            elif txn.isolation_level == IsolationLevel.READ_COMMITTED:
                # Only see committed data (current state)
                return self.tables[table].get(row_id)
            
            elif txn.isolation_level in [IsolationLevel.REPEATABLE_READ, IsolationLevel.SERIALIZABLE]:
                # Use snapshot from transaction start
                return txn.snapshot.get(row_id) if txn.snapshot else None
    
    def write(self, txn: Transaction, table: str, row_id: int, data: Dict[str, Any]):
        """Write a row (uncommitted until commit)."""
        with self.lock:
            txn.write_set.append(row_id)
            if txn.id not in self.uncommitted_writes:
                self.uncommitted_writes[txn.id] = {}
            self.uncommitted_writes[txn.id][row_id] = Row(
                id=row_id, data=data, committed=False, transaction_id=txn.id
            )
    
    def commit(self, txn: Transaction):
        """Commit the transaction."""
        with self.lock:
            if txn.id in self.uncommitted_writes:
                for row_id, row in self.uncommitted_writes[txn.id].items():
                    row.committed = True
                    row.transaction_id = None
                    self.tables["accounts"][row_id] = row
                del self.uncommitted_writes[txn.id]
            
            txn.active = False
            if txn.id in self.active_transactions:
                del self.active_transactions[txn.id]
    
    def rollback(self, txn: Transaction):
        """Rollback the transaction."""
        with self.lock:
            if txn.id in self.uncommitted_writes:
                del self.uncommitted_writes[txn.id]
            txn.active = False
            if txn.id in self.active_transactions:
                del self.active_transactions[txn.id]


print("✅ Database simulation classes loaded!")

---

## 1. Read Uncommitted

The **lowest** isolation level. Transactions can read uncommitted changes from other transactions.

**Characteristics:**
- No read locks acquired
- Can see "dirty" data from uncommitted transactions
- Maximum concurrency, minimum consistency

### Demonstration: Dirty Read

In [ ]:
def demo_dirty_read():
    """
    Demonstrates dirty read phenomenon.
    Transaction 2 reads uncommitted data from Transaction 1,
    which later gets rolled back.
    """
    db = SimpleDatabase()
    db.insert("accounts", 1, {"name": "Alice", "balance": 1000})
    
    results = []
    
    print("=" * 60)
    print("DIRTY READ DEMONSTRATION (Read Uncommitted)")
    print("=" * 60)
    print(f"Initial state: Alice's balance = 1000")
    print()
    
    # Transaction 1: Update but don't commit
    txn1 = db.begin_transaction(IsolationLevel.READ_UNCOMMITTED)
    print(f"[T1] BEGIN TRANSACTION (Read Uncommitted)")
    print(f"[T1] UPDATE accounts SET balance = 2000 WHERE id = 1")
    db.write(txn1, "accounts", 1, {"name": "Alice", "balance": 2000})
    
    # Transaction 2: Read uncommitted data
    txn2 = db.begin_transaction(IsolationLevel.READ_UNCOMMITTED)
    print(f"[T2] BEGIN TRANSACTION (Read Uncommitted)")
    row = db.read(txn2, "accounts", 1)
    print(f"[T2] SELECT balance FROM accounts WHERE id = 1")
    print(f"     → T2 reads balance = {row.data['balance']} (DIRTY READ!)")
    dirty_value = row.data['balance']
    
    # Transaction 1: Rollback
    print(f"[T1] ROLLBACK")
    db.rollback(txn1)
    
    # Check actual value
    actual = db.tables["accounts"][1]
    print(f"\nActual committed balance: {actual.data['balance']}")
    print(f"⚠️  T2 read {dirty_value} but actual value is {actual.data['balance']}!")
    print()
    
    db.commit(txn2)


demo_dirty_read()

---

## 2. Read Committed

The **default** level in most databases (PostgreSQL, Oracle, SQL Server). Only committed data is visible.

**Characteristics:**
- Prevents dirty reads
- Each read sees the latest committed data
- Same row can return different values if read multiple times

### Demonstration: Non-Repeatable Read

In [ ]:
def demo_non_repeatable_read():
    """
    Demonstrates non-repeatable read phenomenon.
    Transaction 1 reads a row twice but gets different values
    because Transaction 2 modified and committed in between.
    """
    db = SimpleDatabase()
    db.insert("accounts", 1, {"name": "Bob", "balance": 500})
    
    print("=" * 60)
    print("NON-REPEATABLE READ DEMONSTRATION (Read Committed)")
    print("=" * 60)
    print(f"Initial state: Bob's balance = 500")
    print()
    
    # Transaction 1: First read
    txn1 = db.begin_transaction(IsolationLevel.READ_COMMITTED)
    print(f"[T1] BEGIN TRANSACTION (Read Committed)")
    row = db.read(txn1, "accounts", 1)
    first_read = row.data['balance']
    print(f"[T1] SELECT balance FROM accounts WHERE id = 1")
    print(f"     → First read: balance = {first_read}")
    
    # Transaction 2: Update and commit
    txn2 = db.begin_transaction(IsolationLevel.READ_COMMITTED)
    print(f"[T2] BEGIN TRANSACTION (Read Committed)")
    print(f"[T2] UPDATE accounts SET balance = 750 WHERE id = 1")
    db.write(txn2, "accounts", 1, {"name": "Bob", "balance": 750})
    print(f"[T2] COMMIT")
    db.commit(txn2)
    
    # Transaction 1: Second read (sees committed change)
    row = db.read(txn1, "accounts", 1)
    second_read = row.data['balance']
    print(f"[T1] SELECT balance FROM accounts WHERE id = 1")
    print(f"     → Second read: balance = {second_read}")
    
    print(f"\n⚠️  NON-REPEATABLE READ: T1 got {first_read} then {second_read}!")
    print(f"    Same query, different results within same transaction.")
    print()
    
    db.commit(txn1)


demo_non_repeatable_read()

---

## 3. Repeatable Read

Ensures that if a transaction reads a row, it will always see the **same value** for that row throughout the transaction.

**Characteristics:**
- Prevents dirty reads and non-repeatable reads
- Uses snapshot isolation (reads from transaction start time)
- Default level in MySQL InnoDB

### Demonstration: Repeatable Read Prevents Non-Repeatable Read

In [ ]:
def demo_repeatable_read():
    """
    Demonstrates how Repeatable Read prevents non-repeatable reads
    by using snapshot isolation.
    """
    db = SimpleDatabase()
    db.insert("accounts", 1, {"name": "Carol", "balance": 300})
    
    print("=" * 60)
    print("REPEATABLE READ DEMONSTRATION")
    print("=" * 60)
    print(f"Initial state: Carol's balance = 300")
    print()
    
    # Transaction 1: Takes a snapshot at start
    txn1 = db.begin_transaction(IsolationLevel.REPEATABLE_READ)
    print(f"[T1] BEGIN TRANSACTION (Repeatable Read)")
    print(f"     📸 Snapshot taken at transaction start")
    row = db.read(txn1, "accounts", 1)
    first_read = row.data['balance']
    print(f"[T1] SELECT balance FROM accounts WHERE id = 1")
    print(f"     → First read: balance = {first_read}")
    
    # Transaction 2: Update and commit
    txn2 = db.begin_transaction(IsolationLevel.REPEATABLE_READ)
    print(f"[T2] BEGIN TRANSACTION (Repeatable Read)")
    print(f"[T2] UPDATE accounts SET balance = 600 WHERE id = 1")
    db.write(txn2, "accounts", 1, {"name": "Carol", "balance": 600})
    print(f"[T2] COMMIT")
    db.commit(txn2)
    
    # Transaction 1: Second read (still sees snapshot)
    row = db.read(txn1, "accounts", 1)
    second_read = row.data['balance']
    print(f"[T1] SELECT balance FROM accounts WHERE id = 1")
    print(f"     → Second read: balance = {second_read}")
    
    print(f"\n✅ CONSISTENT READS: T1 got {first_read} both times!")
    print(f"   Snapshot isolation ensures repeatable reads.")
    print(f"   (Actual committed value is now 600)")
    print()
    
    db.commit(txn1)


demo_repeatable_read()

### Demonstration: Phantom Read (Still Possible in Repeatable Read)

In [ ]:
def demo_phantom_read():
    """
    Demonstrates phantom read phenomenon.
    A transaction runs a query twice and sees new rows
    inserted by another committed transaction.
    
    Note: This is a conceptual simulation. In practice,
    Repeatable Read in some databases (like MySQL InnoDB)
    actually prevents phantoms using next-key locking.
    """
    print("=" * 60)
    print("PHANTOM READ DEMONSTRATION")
    print("=" * 60)
    print()
    
    # Simulated table state
    employees = [
        {"id": 1, "name": "Alice", "department": "Engineering"},
        {"id": 2, "name": "Bob", "department": "Engineering"},
    ]
    
    print("Initial state: 2 employees in Engineering")
    print(f"  → {employees}")
    print()
    
    print("[T1] BEGIN TRANSACTION")
    print("[T1] SELECT COUNT(*) FROM employees WHERE department = 'Engineering'")
    count1 = len([e for e in employees if e['department'] == 'Engineering'])
    print(f"     → Count = {count1}")
    
    print("[T2] BEGIN TRANSACTION")
    print("[T2] INSERT INTO employees VALUES (3, 'Carol', 'Engineering')")
    employees.append({"id": 3, "name": "Carol", "department": "Engineering"})
    print("[T2] COMMIT")
    
    print("[T1] SELECT COUNT(*) FROM employees WHERE department = 'Engineering'")
    count2 = len([e for e in employees if e['department'] == 'Engineering'])
    print(f"     → Count = {count2}")
    
    print(f"\n⚠️  PHANTOM READ: T1 counted {count1} then {count2}!")
    print(f"    A new row 'appeared' during the transaction.")
    print()


demo_phantom_read()

---

## 4. Serializable

The **highest** and most restrictive isolation level. Transactions execute as if they were run sequentially.

**Characteristics:**
- Prevents all concurrency phenomena
- May use locking or optimistic concurrency control
- Transactions may need to retry on conflicts
- Lowest concurrency, highest consistency

### Demonstration: Serializable Prevents Phantoms

In [ ]:
def demo_serializable():
    """
    Demonstrates Serializable isolation preventing phantoms.
    Uses predicate locking or snapshot + conflict detection.
    """
    print("=" * 60)
    print("SERIALIZABLE ISOLATION DEMONSTRATION")
    print("=" * 60)
    print()
    
    print("Serializable ensures transactions appear to execute in sequence.")
    print()
    
    print("Scenario: Two transactions try to transfer money")
    print("─" * 40)
    
    accounts = {"A": 100, "B": 100}
    print(f"Initial: Account A = ${accounts['A']}, Account B = ${accounts['B']}")
    print(f"Total = ${sum(accounts.values())}")
    print()
    
    print("[T1] BEGIN TRANSACTION (Serializable)")
    print("[T1] Read A = 100, B = 100")
    t1_read_a, t1_read_b = accounts["A"], accounts["B"]
    
    print("[T2] BEGIN TRANSACTION (Serializable)")
    print("[T2] Read A = 100, B = 100")
    t2_read_a, t2_read_b = accounts["A"], accounts["B"]
    
    print("[T1] SET A = A + B (transfer all from B to A)")
    print("[T1] SET B = 0")
    # T1 wants: A = 200, B = 0
    
    print("[T2] SET B = A + B (transfer all from A to B)")
    print("[T2] SET A = 0")
    # T2 wants: A = 0, B = 200
    
    print()
    print("⚠️  CONFLICT DETECTED!")
    print("   Both transactions read the same rows and want to modify them.")
    print("   In Serializable mode:")
    print()
    print("   Option 1: T1 commits first, T2 must ABORT and retry")
    print("   Option 2: T2 commits first, T1 must ABORT and retry")
    print()
    
    # Simulate T1 wins
    print("[T1] COMMIT → Success")
    accounts["A"] = t1_read_a + t1_read_b
    accounts["B"] = 0
    print(f"     State: A = ${accounts['A']}, B = ${accounts['B']}")
    
    print("[T2] COMMIT → SERIALIZATION FAILURE (must retry)")
    print()
    print(f"✅ Final state: A = ${accounts['A']}, B = ${accounts['B']}")
    print(f"   Total = ${sum(accounts.values())} (consistency maintained!)")
    print()


demo_serializable()

---

## Comparison: All Isolation Levels

In [ ]:
def print_comparison_table():
    """
    Prints a comprehensive comparison of isolation levels.
    """
    print("╔" + "═" * 78 + "╗")
    print("║" + " TRANSACTION ISOLATION LEVELS COMPARISON ".center(78) + "║")
    print("╠" + "═" * 78 + "╣")
    
    headers = ["Level", "Dirty Read", "Non-Repeatable", "Phantom", "Performance"]
    print("║ {:^18} │ {:^12} │ {:^14} │ {:^10} │ {:^12} ║".format(*headers))
    print("╟" + "─" * 78 + "╢")
    
    data = [
        ["Read Uncommitted", "Possible", "Possible", "Possible", "Fastest"],
        ["Read Committed", "Prevented", "Possible", "Possible", "Fast"],
        ["Repeatable Read", "Prevented", "Prevented", "Possible*", "Moderate"],
        ["Serializable", "Prevented", "Prevented", "Prevented", "Slowest"],
    ]
    
    for row in data:
        formatted = []
        for i, cell in enumerate(row):
            if cell == "Possible" or cell == "Possible*":
                formatted.append(f"⚠️  {cell}")
            elif cell == "Prevented":
                formatted.append(f"✅ {cell}")
            else:
                formatted.append(cell)
        print("║ {:^18} │ {:^12} │ {:^14} │ {:^10} │ {:^12} ║".format(*formatted))
    
    print("╚" + "═" * 78 + "╝")
    print("\n* MySQL InnoDB prevents phantoms at Repeatable Read using next-key locking")


print_comparison_table()

---

## When to Use Each Isolation Level

### Read Uncommitted
| Use Case | Trade-off |
|----------|----------|
| Approximate counts/analytics | Data may be inconsistent |
| Monitoring dashboards | Dirty reads possible |
| Non-critical reporting | No integrity guarantees |

### Read Committed (Default for most DBs)
| Use Case | Trade-off |
|----------|----------|
| General OLTP workloads | Non-repeatable reads |
| Web applications | May need retry logic |
| Session-based operations | Good balance of safety/speed |

### Repeatable Read
| Use Case | Trade-off |
|----------|----------|
| Financial reports (point-in-time) | Phantom reads possible |
| Inventory snapshots | Higher memory usage |
| Batch processing needing consistency | Longer lock holding |

### Serializable
| Use Case | Trade-off |
|----------|----------|
| Bank transfers | Lowest throughput |
| Booking systems (no double-booking) | Frequent retries needed |
| Regulatory compliance | Deadlock risk |
| Critical financial transactions | Highest latency |

---

## Database-Specific Implementation Notes

In [ ]:
database_notes = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                    DATABASE-SPECIFIC IMPLEMENTATION NOTES                     ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  PostgreSQL                                                                  ║
║  ──────────                                                                  ║
║  • Default: Read Committed                                                   ║
║  • Uses MVCC (Multi-Version Concurrency Control)                            ║
║  • Serializable uses SSI (Serializable Snapshot Isolation)                  ║
║  • SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;                            ║
║                                                                              ║
║  MySQL (InnoDB)                                                              ║
║  ─────────────                                                               ║
║  • Default: Repeatable Read                                                  ║
║  • Uses MVCC + next-key locking                                              ║
║  • Repeatable Read actually prevents phantoms!                               ║
║  • SET SESSION TRANSACTION ISOLATION LEVEL READ COMMITTED;                   ║
║                                                                              ║
║  SQL Server                                                                  ║
║  ──────────                                                                  ║
║  • Default: Read Committed                                                   ║
║  • Offers SNAPSHOT isolation (similar to Repeatable Read)                   ║
║  • Uses locking by default, MVCC optional                                   ║
║  • SET TRANSACTION ISOLATION LEVEL SNAPSHOT;                                ║
║                                                                              ║
║  Oracle                                                                      ║
║  ──────────                                                                  ║
║  • Default: Read Committed                                                   ║
║  • Only supports Read Committed and Serializable                            ║
║  • Uses MVCC (undo segments)                                                 ║
║  • SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;                            ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""
print(database_notes)

---

## SQL Examples for Setting Isolation Levels

In [ ]:
sql_examples = """
-- PostgreSQL / MySQL / SQL Server (ANSI SQL)
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SET TRANSACTION ISOLATION LEVEL READ COMMITTED;
SET TRANSACTION ISOLATION LEVEL REPEATABLE READ;
SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;

-- PostgreSQL: Per-transaction
BEGIN TRANSACTION ISOLATION LEVEL SERIALIZABLE;
    SELECT * FROM accounts WHERE id = 1;
    UPDATE accounts SET balance = balance - 100 WHERE id = 1;
COMMIT;

-- MySQL: Session-level
SET SESSION TRANSACTION ISOLATION LEVEL REPEATABLE READ;
START TRANSACTION;
    SELECT * FROM accounts WHERE id = 1 FOR UPDATE;  -- Explicit locking
    UPDATE accounts SET balance = balance - 100 WHERE id = 1;
COMMIT;

-- SQL Server: With SNAPSHOT
ALTER DATABASE MyDB SET ALLOW_SNAPSHOT_ISOLATION ON;
SET TRANSACTION ISOLATION LEVEL SNAPSHOT;
BEGIN TRANSACTION;
    SELECT * FROM accounts WHERE id = 1;
    -- Sees consistent snapshot, no blocking
COMMIT;
"""

print("SQL EXAMPLES FOR SETTING ISOLATION LEVELS")
print("=" * 50)
print(sql_examples)

---

## 📌 Key Takeaways

### The Trade-off Triangle

```
         Consistency
             △
            /│\
           / │ \
          /  │  \
         /   │   \
        ─────┼─────
    Concurrency ◁───▷ Performance
```

### Summary Points

1. **Choose the right level for your use case** - Don't default to Serializable "just to be safe"

2. **Understand your database's implementation** - MySQL's Repeatable Read ≠ PostgreSQL's Repeatable Read

3. **Design for retries** - Higher isolation = more transaction failures

4. **Use explicit locking when needed** - `SELECT ... FOR UPDATE` can provide guarantees at lower isolation

5. **Test concurrent scenarios** - Isolation bugs only appear under load

### Quick Decision Guide

| Scenario | Recommended Level |
|----------|------------------|
| Analytics/Reporting (approx.) | Read Uncommitted |
| Standard web app | Read Committed |
| Financial reporting | Repeatable Read |
| Money transfers | Serializable |
| Inventory management | Repeatable Read + Explicit Locks |

---

**Remember**: The goal is to use the *lowest* isolation level that still meets your consistency requirements!